In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer

df = pd.read_csv("../data/df.csv", index_col=False)


numerical_features_full = [
    "n_items", "total_price", "total_freight", "avg_price",
    "payment_value", "seller_avg_score_past",
    "purchase_to_estimated_days",
]

numerical_features_reduced = [
    "n_items", "total_price", "total_freight", "avg_price",
    "payment_value",
    "purchase_to_estimated_days",
]

ordinal_linear = ["payment_installments"]
ordinal_cyclic = ["purchase_month", "purchase_dow"]
categorical_features = ["main_payment_type", "same_state", "customer_state", "seller_state"]

X = df.drop(columns=["is_late"])
y = df["is_late"]
groups = df["customer_unique_id"]
def encode_cyclic_features(df_sub):
    df_sub = df_sub.copy()
    return pd.DataFrame({
        "month_sin": np.sin(2 * np.pi * df_sub["purchase_month"] / 12),
        "month_cos": np.cos(2 * np.pi * df_sub["purchase_month"] / 12),
        "dow_sin": np.sin(2 * np.pi * df_sub["purchase_dow"] / 7),
        "dow_cos": np.cos(2 * np.pi * df_sub["purchase_dow"] / 7),
    })

preprocessor_full = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features_full + ordinal_linear),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ("cyclic", FunctionTransformer(encode_cyclic_features), ordinal_cyclic),
    ]
)

preprocessor_reduced = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features_reduced + ordinal_linear),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ("cyclic", FunctionTransformer(encode_cyclic_features), ordinal_cyclic),
    ]
)

print("Two preprocessors defined: preprocessor_full & preprocessor_reduced")

Train+Val size: (78254, 15), Test size: (19564, 15)
Two preprocessors defined: preprocessor_full & preprocessor_reduced


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score
)
from sklearn.base import clone
from tqdm import tqdm
import joblib
import numpy as np

n_runs = 3
base_seed = 42

test_f1_list_full = []            
best_global_test_f1_full = -1.0  
best_run_idx_full = None

best_preprocessor_full = None
best_model_full = None
best_params_full = None
best_val_f1_full = None
best_X_test_full_pre = None
best_y_test_full = None

for i in range(1, n_runs + 1):
    print(f"\n================== LogReg FULL Run {i} / {n_runs} ==================")
    seed = base_seed * i
    X_other, X_test, y_other, y_test = train_test_split(X, y,test_size=0.2,random_state=seed,stratify=y)

    mask_full_other = X_other["seller_avg_score_past"].notna()
    mask_full_test  = X_test["seller_avg_score_past"].notna()

    X_other_full = X_other[mask_full_other].copy()
    y_other_full = y_other[mask_full_other].copy()
    X_test_full  = X_test[mask_full_test].copy()
    y_test_full  = y_test[mask_full_test].copy()

    print(f"[Run {i}][FULL]   Train+Val rows (no missing seller_avg_score_past): {X_other_full.shape[0]}")
    print(f"[Run {i}][FULL]   Test rows (no missing seller_avg_score_past):      {X_test_full.shape[0]}")

    X_train_full_raw, X_val_full_raw, y_train_full, y_val_full = train_test_split(X_other_full, y_other_full,test_size=0.25,random_state=seed,stratify=y_other_full)

    preproc_this_run_full = clone(preprocessor_full)
    preproc_this_run_full.fit(X_train_full_raw)

    X_train_full    = preproc_this_run_full.transform(X_train_full_raw)
    X_val_full      = preproc_this_run_full.transform(X_val_full_raw)
    X_test_full_pre = preproc_this_run_full.transform(X_test_full)

    print("\n=== FULL-FEATURE (no missing) ===")
    print(f"[Run {i}] X_train_full shape:", X_train_full.shape)
    print(f"[Run {i}] X_val_full   shape:", X_val_full.shape)
    print(f"[Run {i}] X_test_full  shape:", X_test_full_pre.shape)
    print(f"[Run {i}] NaNs in X_train_full:", np.isnan(X_train_full).sum())

    param_grid_full = {
    "C": [0.01, 0.1, 1, 10],
    "class_weight": [None, "balanced"],
    "penalty": ["l2", "l1"],
    "solver": ["liblinear"], 
    }
    param_list_full = list(ParameterGrid(param_grid_full))
    print(f"\n[LogReg FULL][Run {i}] Total param combinations: {len(param_list_full)}")

    best_full_model_run  = None
    best_full_params_run = None
    best_full_f1_run     = -1.0

    for params in tqdm(param_list_full, desc=f"Searching LogReg (FULL) - run {i}"):
        model = LogisticRegression(
            max_iter=1000,
            **params,
        )
        model.fit(X_train_full, y_train_full)

        y_val_pred_full = model.predict(X_val_full)
        f1_full = f1_score(y_val_full, y_val_pred_full)

        if f1_full > best_full_f1_run:
            best_full_f1_run     = f1_full
            best_full_params_run = params
            best_full_model_run  = model

    print("\n[Logistic Regression FULL]")
    print(f"[Run {i}] Best params (FULL):", best_full_params_run)
    print(f"[Run {i}] Best Val F1 (label=1, FULL): {best_full_f1_run:.4f}")

    y_test_full_pred = best_full_model_run.predict(X_test_full_pre)

    test_f1_full   = f1_score(y_test_full, y_test_full_pred)
    test_acc_full  = accuracy_score(y_test_full, y_test_full_pred)
    test_prec_full = precision_score(y_test_full, y_test_full_pred, zero_division=0)
    test_rec_full  = recall_score(y_test_full, y_test_full_pred, zero_division=0)
    cm_full        = confusion_matrix(y_test_full, y_test_full_pred)
    proba_full     = best_full_model_run.predict_proba(X_test_full_pre)[:, 1]
    auc_full       = roc_auc_score(y_test_full, proba_full)

    print(f"\n[LogReg FULL][Run {i}] Test F1 (label=1): {test_f1_full:.4f}")
    test_f1_list_full.append(test_f1_full)

    if test_f1_full > best_global_test_f1_full:
        best_global_test_f1_full = test_f1_full
        best_run_idx_full        = i
        best_preprocessor_full   = preproc_this_run_full
        best_model_full          = best_full_model_run
        best_params_full         = best_full_params_run
        best_val_f1_full         = best_full_f1_run
        best_X_test_full_pre     = X_test_full_pre
        best_y_test_full         = y_test_full

        print("\n--> New global BEST LogReg FULL run so far!")
        print("[Global best LogReg FULL so far] Test metrics:")
        print(f"Accuracy : {test_acc_full:.4f}")
        print(f"Precision: {test_prec_full:.4f}")
        print(f"Recall   : {test_rec_full:.4f}")
        print(f"F1       : {test_f1_full:.4f}")
        print(f"ROC-AUC  : {auc_full:.4f}")
        print("Confusion matrix [[TN, FP], [FN, TP]]:")
        print(cm_full)
        print("\nClassification report:")
        print(classification_report(y_test_full, y_test_full_pred))

test_f1_array_full = np.array(test_f1_list_full)
print("\n================== LogReg FULL Summary over runs ==================")
print("LogReg FULL Test F1 for each run:", test_f1_list_full)
print(f"Mean LogReg FULL Test F1 over {n_runs} runs: {test_f1_array_full.mean():.4f}")
print(f"Std  LogReg FULL Test F1 over {n_runs} runs: {test_f1_array_full.std():.4f}")
print(f"Best LogReg FULL run index: {best_run_idx_full}, Best Test F1: {best_global_test_f1_full:.4f}")

y_best_test_full_pred = best_model_full.predict(best_X_test_full_pre)
best_test_f1_full   = f1_score(best_y_test_full, y_best_test_full_pred)
best_test_acc_full  = accuracy_score(best_y_test_full, y_best_test_full_pred)
best_test_prec_full = precision_score(best_y_test_full, y_best_test_full_pred, zero_division=0)
best_test_rec_full  = recall_score(best_y_test_full, y_best_test_full_pred, zero_division=0)
best_cm_full        = confusion_matrix(best_y_test_full, y_best_test_full_pred)
best_proba_full     = best_model_full.predict_proba(best_X_test_full_pre)[:, 1]
best_auc_full       = roc_auc_score(best_y_test_full, best_proba_full)

print("\n================== Final best LogReg FULL run metrics ==================")
print(f"[Best LogReg FULL run #{best_run_idx_full}] Test metrics:")
print(f"Accuracy : {best_test_acc_full:.4f}")
print(f"Precision: {best_test_prec_full:.4f}")
print(f"Recall   : {best_test_rec_full:.4f}")
print(f"F1       : {best_test_f1_full:.4f}")
print(f"ROC-AUC  : {best_auc_full:.4f}")
print("Confusion matrix [[TN, FP], [FN, TP]]:")
print(best_cm_full)
print("\nClassification report:")
print(classification_report(best_y_test_full, y_best_test_full_pred))
lr_full_artifacts = {
    "preprocessor_full": best_preprocessor_full,
    "model_full": best_model_full,
    "best_params_full": best_params_full,
    "best_val_f1_full": best_val_f1_full,
    "best_test_f1_full": best_global_test_f1_full,
    "test_f1_all_runs_full": test_f1_list_full,
}

joblib.dump(lr_full_artifacts, "best_logreg_full_model_3_run.pkl")
print("\nSaved FULL-feature Logistic Regression (best of multi-run) → best_logreg_full_model_3_run.pkl")


================== LogReg FULL Run 1 / 3 ==================
[Run 1][FULL]   Train+Val rows (no missing seller_avg_score_past): 73486
[Run 1][FULL]   Test rows (no missing seller_avg_score_past):      18394

=== FULL-FEATURE (no missing) ===
[Run 1] X_train_full shape: (55114, 66)
[Run 1] X_val_full   shape: (18372, 66)
[Run 1] X_test_full  shape: (18394, 66)
[Run 1] NaNs in X_train_full: 0

[LogReg FULL][Run 1] Total param combinations: 16


Searching LogReg (FULL) - run 1: 100%|██████████| 16/16 [01:17<00:00,  4.84s/it]



[Logistic Regression FULL]
[Run 1] Best params (FULL): {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
[Run 1] Best Val F1 (label=1, FULL): 0.2457

[LogReg FULL][Run 1] Test F1 (label=1): 0.2516

--> New global BEST LogReg FULL run so far!
[Global best LogReg FULL so far] Test metrics:
Accuracy : 0.6627
Precision: 0.1530
Recall   : 0.7086
F1       : 0.2516
ROC-AUC  : 0.7476
Confusion matrix [[TN, FP], [FN, TP]]:
[[11147  5775]
 [  429  1043]]

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.66      0.78     16922
           1       0.15      0.71      0.25      1472

    accuracy                           0.66     18394
   macro avg       0.56      0.68      0.52     18394
weighted avg       0.90      0.66      0.74     18394


================== LogReg FULL Run 2 / 3 ==================
[Run 2][FULL]   Train+Val rows (no missing seller_avg_score_past): 73587
[Run 2][FULL]   Test rows (no missin

Searching LogReg (FULL) - run 2: 100%|██████████| 16/16 [05:42<00:00, 21.42s/it]



[Logistic Regression FULL]
[Run 2] Best params (FULL): {'C': 1, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}
[Run 2] Best Val F1 (label=1, FULL): 0.2517

[LogReg FULL][Run 2] Test F1 (label=1): 0.2488

================== LogReg FULL Run 3 / 3 ==================
[Run 3][FULL]   Train+Val rows (no missing seller_avg_score_past): 73482
[Run 3][FULL]   Test rows (no missing seller_avg_score_past):      18398

=== FULL-FEATURE (no missing) ===
[Run 3] X_train_full shape: (55111, 66)
[Run 3] X_val_full   shape: (18371, 66)
[Run 3] X_test_full  shape: (18398, 66)
[Run 3] NaNs in X_train_full: 0

[LogReg FULL][Run 3] Total param combinations: 16


Searching LogReg (FULL) - run 3: 100%|██████████| 16/16 [01:23<00:00,  5.24s/it]


[Logistic Regression FULL]
[Run 3] Best params (FULL): {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
[Run 3] Best Val F1 (label=1, FULL): 0.2484

[LogReg FULL][Run 3] Test F1 (label=1): 0.2444

================== LogReg FULL Summary over runs ==================
LogReg FULL Test F1 for each run: [0.25162846803377564, 0.24883750157094384, 0.24439083232810616]
Mean LogReg FULL Test F1 over 3 runs: 0.2483
Std  LogReg FULL Test F1 over 3 runs: 0.0030
Best LogReg FULL run index: 1, Best Test F1: 0.2516

================== Final best LogReg FULL run metrics ==================
[Best LogReg FULL run #1] Test metrics:
Accuracy : 0.6627
Precision: 0.1530
Recall   : 0.7086
F1       : 0.2516
ROC-AUC  : 0.7476
Confusion matrix [[TN, FP], [FN, TP]]:
[[11147  5775]
 [  429  1043]]

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.66      0.78     16922
           1       0.15      0.71      0.25      1472

  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score
)
from sklearn.base import clone
from tqdm import tqdm
import joblib
import numpy as np

n_runs = 3
base_seed = 42

test_f1_list_miss = []             
best_global_test_f1_miss = -1.0     
best_run_idx_miss = None

best_preprocessor_miss = None
best_model_miss = None
best_params_miss = None
best_val_f1_miss = None
best_X_test_miss_pre = None
best_y_test_miss = None

for i in range(1, n_runs + 1):
    print(f"\n================== LogReg MISSING Run {i} / {n_runs} ==================")
    seed = base_seed * i

    X_other, X_test, y_other, y_test = train_test_split(X, y,test_size=0.2,random_state=seed,stratify=y)
    mask_miss_other = X_other["seller_avg_score_past"].isna()
    mask_miss_test  = X_test["seller_avg_score_past"].isna()

    X_other_miss = X_other[mask_miss_other].copy()
    y_other_miss = y_other[mask_miss_other].copy()
    X_test_miss  = X_test[mask_miss_test].copy()
    y_test_miss  = y_test[mask_miss_test].copy()

    print(f"[Run {i}][MISSING] Train+Val rows (missing seller_avg_score_past): {X_other_miss.shape[0]}")
    print(f"[Run {i}][MISSING] Test rows (missing seller_avg_score_past):      {X_test_miss.shape[0]}")


    if X_other_miss.shape[0] < 100 or X_test_miss.shape[0] < 50:
        print(f"[Run {i}] Too few missing samples, skip this run.")
        continue


    X_train_miss_raw, X_val_miss_raw, y_train_miss, y_val_miss = train_test_split(X_other_miss, y_other_miss,test_size=0.25,random_state=seed,stratify=y_other_miss)

    preproc_this_run_miss = clone(preprocessor_reduced)
    preproc_this_run_miss.fit(X_train_miss_raw)

    X_train_miss    = preproc_this_run_miss.transform(X_train_miss_raw)
    X_val_miss      = preproc_this_run_miss.transform(X_val_miss_raw)
    X_test_miss_pre = preproc_this_run_miss.transform(X_test_miss)

    print("\n=== REDUCED-FEATURE (MISSING seller_avg_score_past) ===")
    print(f"[Run {i}] X_train_miss shape:", X_train_miss.shape)
    print(f"[Run {i}] X_val_miss   shape:", X_val_miss.shape)
    print(f"[Run {i}] X_test_miss  shape:", X_test_miss_pre.shape)
    print(f"[Run {i}] NaNs in X_train_miss:", np.isnan(X_train_miss).sum())

    param_grid_miss = {
    "C": [0.01, 0.1, 1, 10],
    "class_weight": [None, "balanced"],
    "penalty": ["l2", "l1"],
    "solver": ["liblinear"], 
    }
    param_list_miss = list(ParameterGrid(param_grid_miss))
    print(f"\n[LogReg MISSING][Run {i}] Total param combinations: {len(param_list_miss)}")

    best_miss_model_run  = None
    best_miss_params_run = None
    best_miss_f1_run     = -1.0

    for params in tqdm(param_list_miss, desc=f"Searching LogReg (MISSING) - run {i}"):
        model = LogisticRegression(
            max_iter=1000,
            **params,
        )
        model.fit(X_train_miss, y_train_miss)

        y_val_pred_miss = model.predict(X_val_miss)
        f1_miss = f1_score(y_val_miss, y_val_pred_miss)

        if f1_miss > best_miss_f1_run:
            best_miss_f1_run     = f1_miss
            best_miss_params_run = params
            best_miss_model_run  = model

    print("\n[Logistic Regression MISSING]")
    print(f"[Run {i}] Best params (MISSING):", best_miss_params_run)
    print(f"[Run {i}] Best Val F1 (label=1, MISSING): {best_miss_f1_run:.4f}")

    y_test_miss_pred = best_miss_model_run.predict(X_test_miss_pre)

    test_f1_miss   = f1_score(y_test_miss, y_test_miss_pred)
    test_acc_miss  = accuracy_score(y_test_miss, y_test_miss_pred)
    test_prec_miss = precision_score(y_test_miss, y_test_miss_pred, zero_division=0)
    test_rec_miss  = recall_score(y_test_miss, y_test_miss_pred, zero_division=0)
    cm_miss        = confusion_matrix(y_test_miss, y_test_miss_pred)
    proba_miss     = best_miss_model_run.predict_proba(X_test_miss_pre)[:, 1]
    auc_miss       = roc_auc_score(y_test_miss, proba_miss)

    print(f"\n[LogReg MISSING][Run {i}] Test F1 (label=1): {test_f1_miss:.4f}")
    test_f1_list_miss.append(test_f1_miss)

    if test_f1_miss > best_global_test_f1_miss:
        best_global_test_f1_miss = test_f1_miss
        best_run_idx_miss        = i
        best_preprocessor_miss   = preproc_this_run_miss
        best_model_miss          = best_miss_model_run
        best_params_miss         = best_miss_params_run
        best_val_f1_miss         = best_miss_f1_run
        best_X_test_miss_pre     = X_test_miss_pre
        best_y_test_miss         = y_test_miss

        print("\n--> New global BEST LogReg MISSING run so far!")
        print("[Global best LogReg MISSING so far] Test metrics:")
        print(f"Accuracy : {test_acc_miss:.4f}")
        print(f"Precision: {test_prec_miss:.4f}")
        print(f"Recall   : {test_rec_miss:.4f}")
        print(f"F1       : {test_f1_miss:.4f}")
        print(f"ROC-AUC  : {auc_miss:.4f}")
        print("Confusion matrix [[TN, FP], [FN, TP]]:")
        print(cm_miss)
        print("\nClassification report:")
        print(classification_report(y_test_miss, y_test_miss_pred))

test_f1_array_miss = np.array(test_f1_list_miss)
print("\n================== LogReg MISSING Summary over runs ==================")
print("LogReg MISSING Test F1 for each run:", test_f1_list_miss)
if len(test_f1_array_miss) > 0:
    print(f"Mean LogReg MISSING Test F1 over {len(test_f1_array_miss)} runs: {test_f1_array_miss.mean():.4f}")
    print(f"Std  LogReg MISSING Test F1 over {len(test_f1_array_miss)} runs: {test_f1_array_miss.std():.4f}")
    print(f"Best LogReg MISSING run index: {best_run_idx_miss}, Best Test F1: {best_global_test_f1_miss:.4f}")
else:
    print("No valid runs (too few missing samples in all runs).")

if best_model_miss is not None:
    y_best_test_miss_pred = best_model_miss.predict(best_X_test_miss_pre)
    best_test_f1_miss   = f1_score(best_y_test_miss, y_best_test_miss_pred)
    best_test_acc_miss  = accuracy_score(best_y_test_miss, y_best_test_miss_pred)
    best_test_prec_miss = precision_score(best_y_test_miss, y_best_test_miss_pred, zero_division=0)
    best_test_rec_miss  = recall_score(best_y_test_miss, y_best_test_miss_pred, zero_division=0)
    best_cm_miss        = confusion_matrix(best_y_test_miss, y_best_test_miss_pred)
    best_proba_miss     = best_model_miss.predict_proba(best_X_test_miss_pre)[:, 1]
    best_auc_miss       = roc_auc_score(best_y_test_miss, best_proba_miss)

    print("\n================== Final best LogReg MISSING run metrics ==================")
    print(f"[Best LogReg MISSING run #{best_run_idx_miss}] Test metrics:")
    print(f"Accuracy : {best_test_acc_miss:.4f}")
    print(f"Precision: {best_test_prec_miss:.4f}")
    print(f"Recall   : {best_test_rec_miss:.4f}")
    print(f"F1       : {best_test_f1_miss:.4f}")
    print(f"ROC-AUC  : {best_auc_miss:.4f}")
    print("Confusion matrix [[TN, FP], [FN, TP]]:")
    print(best_cm_miss)
    print("\nClassification report:")
    print(classification_report(best_y_test_miss, y_best_test_miss_pred))

    lr_miss_artifacts = {
        "preprocessor_reduced": best_preprocessor_miss,
        "model_miss": best_model_miss,
        "best_params_miss": best_params_miss,
        "best_val_f1_miss": best_val_f1_miss,
        "best_test_f1_miss": best_global_test_f1_miss,
        "test_f1_all_runs_miss": test_f1_list_miss,
    }

    joblib.dump(lr_miss_artifacts, "best_logreg_missing_reduced_model_3_run.pkl")
    print("\nSaved MISSING / reduced-feature Logistic Regression (best of multi-run) → best_logreg_missing_reduced_model_3_run.pkl")
else:
    print("\nNo best MISSING model to save (maybe every run was skipped due to too few missing samples).")


================== LogReg MISSING Run 1 / 3 ==================
[Run 1][MISSING] Train+Val rows (missing seller_avg_score_past): 4768
[Run 1][MISSING] Test rows (missing seller_avg_score_past):      1170

=== REDUCED-FEATURE (MISSING seller_avg_score_past) ===
[Run 1] X_train_miss shape: (3576, 64)
[Run 1] X_val_miss   shape: (1192, 64)
[Run 1] X_test_miss  shape: (1170, 64)
[Run 1] NaNs in X_train_miss: 0

[LogReg MISSING][Run 1] Total param combinations: 4


Searching LogReg (MISSING) - run 1: 100%|██████████| 4/4 [00:00<00:00, 42.46it/s]


[Logistic Regression MISSING]
[Run 1] Best params (MISSING): {'C': 1, 'class_weight': 'balanced'}
[Run 1] Best Val F1 (label=1, MISSING): 0.2505

[LogReg MISSING][Run 1] Test F1 (label=1): 0.2461

--> New global BEST LogReg MISSING run so far!
[Global best LogReg MISSING so far] Test metrics:
Accuracy : 0.6701
Precision: 0.1518
Recall   : 0.6495
F1       : 0.2461
ROC-AUC  : 0.7419
Confusion matrix [[TN, FP], [FN, TP]]:
[[721 352]
 [ 34  63]]

Classification report:
              precision    recall  f1-score   support

           0       0.95      0.67      0.79      1073
           1       0.15      0.65      0.25        97

    accuracy                           0.67      1170
   macro avg       0.55      0.66      0.52      1170
weighted avg       0.89      0.67      0.74      1170


================== LogReg MISSING Run 2 / 3 ==================


[Run 2][MISSING] Train+Val rows (missing seller_avg_score_past): 4667
[Run 2][MISSING] Test rows (missing seller_avg_score_past):      1271

=== REDUCED-FEATURE (MISSING seller_avg_score_past) ===
[Run 2] X_train_miss shape: (3500, 64)
[Run 2] X_val_miss   shape: (1167, 64)
[Run 2] X_test_miss  shape: (1271, 64)
[Run 2] NaNs in X_train_miss: 0

[LogReg MISSING][Run 2] Total param combinations: 4


Searching LogReg (MISSING) - run 2: 100%|██████████| 4/4 [00:00<00:00, 44.99it/s]



[Logistic Regression MISSING]
[Run 2] Best params (MISSING): {'C': 0.01, 'class_weight': 'balanced'}
[Run 2] Best Val F1 (label=1, MISSING): 0.2734

[LogReg MISSING][Run 2] Test F1 (label=1): 0.2791

--> New global BEST LogReg MISSING run so far!
[Global best LogReg MISSING so far] Test metrics:
Accuracy : 0.6341
Precision: 0.1708
Recall   : 0.7627
F1       : 0.2791
ROC-AUC  : 0.7613
Confusion matrix [[TN, FP], [FN, TP]]:
[[716 437]
 [ 28  90]]

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.62      0.75      1153
           1       0.17      0.76      0.28       118

    accuracy                           0.63      1271
   macro avg       0.57      0.69      0.52      1271
weighted avg       0.89      0.63      0.71      1271


================== LogReg MISSING Run 3 / 3 ==================
[Run 3][MISSING] Train+Val rows (missing seller_avg_score_past): 4772
[Run 3][MISSING] Test rows (missing seller_avg_score_past):      

Searching LogReg (MISSING) - run 3: 100%|██████████| 4/4 [00:00<00:00, 41.60it/s]


[Logistic Regression MISSING]
[Run 3] Best params (MISSING): {'C': 10, 'class_weight': 'balanced'}
[Run 3] Best Val F1 (label=1, MISSING): 0.2693

[LogReg MISSING][Run 3] Test F1 (label=1): 0.3055

--> New global BEST LogReg MISSING run so far!
[Global best LogReg MISSING so far] Test metrics:
Accuracy : 0.6724
Precision: 0.1991
Recall   : 0.6562
F1       : 0.3055
ROC-AUC  : 0.7303
Confusion matrix [[TN, FP], [FN, TP]]:
[[700 338]
 [ 44  84]]

Classification report:
              precision    recall  f1-score   support

           0       0.94      0.67      0.79      1038
           1       0.20      0.66      0.31       128

    accuracy                           0.67      1166
   macro avg       0.57      0.67      0.55      1166
weighted avg       0.86      0.67      0.73      1166


================== LogReg MISSING Summary over runs ==================
LogReg MISSING Test F1 for each run: [0.24609375, 0.27906976744186046, 0.3054545454545455]
Mean LogReg MISSING Test F1 over 3 run